In [1]:
from os import walk
from joblib import Parallel, delayed
import librosa
import numpy as np
import multiprocessing
import json
from deep_audio import Directory, Audio, JSON

In [2]:
num_cores = multiprocessing.cpu_count()

sampling_rate = 24000

path = f'audios/{sampling_rate}'

f = Directory.filenames(path)

data = {
    "mapping": [],
    "lpcc": [],
    "labels": []
}

In [3]:
def process_directory(dir, index):
    signal, sr = Audio.read(f'{path}/{dir}', sr=sampling_rate, normalize=False)

    signal = np.array(signal)

    signal = signal[:len(signal) - len(signal) % (sr * 5)]

    segments = len(signal) // (sr * 5)

    m = {
        "lpcc": [],
        "labels": [index] * segments
    }

    for i in range(segments):
        start_sample = sr * i * 5
        finish_sample = start_sample + (sr * 5)

        sample = signal[start_sample:finish_sample]

        lpcc = Audio.lpcc(sample, rate=sr)
        lpcc = lpcc * 100
        lpcc = lpcc.T

        m['lpcc'].append(lpcc.tolist())

    print(f'{dir} -> segments: {segments}')
    return m

In [4]:
def object_lpcc_to_json(m):
    data['mapping'] = [file.replace('.wav', '') for file in f]

    for i in m:
        data['lpcc'].extend(i['lpcc'])
        data['labels'].extend(i['labels'])

    JSON.create_json_file(f'processed/lpcc/lpcc_{sampling_rate}.json', data)


In [5]:
if __name__ == '__main__':
    m = Parallel(n_jobs=num_cores, verbose=len(f), temp_folder='./tmp/')(
        delayed(process_directory)(i, j) for j, i in enumerate(f) if i is not None)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:   17.3s
[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed:   25.2s
[Parallel(n_jobs=4)]: Done   3 tasks      | elapsed:   25.3s
[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed:   26.6s
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:   37.2s
[Parallel(n_jobs=4)]: Done   6 tasks      | elapsed:   48.0s
[Parallel(n_jobs=4)]: Done   7 tasks      | elapsed:   51.0s
[Parallel(n_jobs=4)]: Done   8 tasks      | elapsed:   54.5s
[Parallel(n_jobs=4)]: Done   9 tasks      | elapsed:  1.1min
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:  1.2min
[Parallel(n_jobs=4)]: Done  11 tasks      | elapsed:  1.3min
[Parallel(n_jobs=4)]: Done  12 tasks      | elapsed:  1.3min
[Parallel(n_jobs=4)]: Done  13 tasks      | elapsed:  1.5min
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed:  1.5min
[Parallel(n_jobs=4)]: Done  15 tasks      | elapsed:  1.6min
[Parallel(

In [6]:
    object_lpcc_to_json(m)